In [ ]:
# Debug level
debug=2

#Input files
backbone_weights='trained_backbone.ckpt'
rpn_weights='trained_rpn.ckpt'
class_weights='trained_classifier.ckpt'
datapath='tensorflow-great-barrier-reef'

In [ ]:
# Boilerplate
import sys
import numpy as np
from matplotlib.pyplot import figure, imshow, gca, tight_layout, show
from matplotlib import patches
from importlib import reload
import tensorflow as tf
import tensorflow_addons as tfa
import os

if 'jointmodel' not in sys.modules:
    import backbone
    import classifier
    import data_utils
    import faster_rcnn
    import rpn
    import roi_pooling
    import geometry
    import evaluation
    import callback
    import jointmodel
else:
    reload(backbone)
    reload(classifier)
    reload(data_utils)
    reload(faster_rcnn)
    reload(rpn)
    reload(roi_pooling)
    reload(geometry)
    reload(evaluation)
    reload(callback)
    reload(jointmodel)

In [ ]:
# Instantiate everything from saved weights
frcnn = faster_rcnn.FasterRCNNWrapper(
    input_shape=(720, 1280, 3),
    datapath=datapath,
    backbone_type='ResNet50',
    backbone_weights=backbone_weights,
    rpn_weights=rpn_weights,
    classifier_weights= class_weights,
    positive_threshold=0.0,
    debug=debug
)

In [ ]:
# Test that the RPN components work as expected
training = frcnn.data_loader_full.get_validation().__iter__()

In [ ]:
# Optimizer uses the weight decay and learning rates
# from the final base training epoch. Uses the
# same clip value and momentum parameters as the classifier
# although the gradient clip shouldn't matter here.
fine_optimizer = tfa.optimizers.SGDW(
            learning_rate=frcnn.classifier_learning_rate["values"][-1], # This is almost certainly too slow
            weight_decay=frcnn.classifier_weight_decay["values"][-1], # This is also too slow
            momentum=frcnn.classifier_momentum,
            clipvalue=frcnn.classifier_clipvalue,
)

# Important - free up the backbone weights
frcnn.backbone.set_trainable(True)

frcnn.rpnwrapper.rpnmodel._positive.assign(500.)
frcnn.rpnwrapper.rpnmodel._negative.assign(100.)
frcnn.classmodel._positive.assign(500.)
frcnn.classmodel._negative.assign(100.)

In [ ]:
# Make the joint model
reload(jointmodel)

joint_model = jointmodel.JointModel(
            frcnn.backbone,
            frcnn.rpnwrapper.rpnmodel,

            frcnn.data_loader_full.decode_label,
            frcnn.classifier_augmentation
)

joint_model.optimizer = fine_optimizer
joint_model.compiled_metrics = [
                evaluation.TopNRegionsRecall(
                    self.rpnwrapper.top_n_recall,
                    self.data_loader_full.decode_label,
                    name="top%d_recall" % self.rpnwrapper.top_n_recall,
                )
            ]

In [ ]:
joint_model.train_step(training.next())

In [ ]:
joint_model.test_step(training.next())